In [1]:
import keras

Using TensorFlow backend.


In [2]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras import regularizers

In [3]:
# check if tpu is running
import tensorflow as tf
import os
import pprint

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!


In [7]:
running_on_colab = True
if running_on_colab:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
else:
    (x_train, y_train), (x_test, y_test) =  tf.keras.datasets.fashion_mnist.load_data()

# Scales the training and test data to range between 0 and 1.
max_value = float(x_train.max())
x_train = x_train.astype('float32') / max_value
x_test = x_test.astype('float32') / max_value

In [14]:
x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [9]:
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [10]:
(x_train.shape, x_test.shape)

((60000, 784), (10000, 784))

In [11]:
input_dim = x_train.shape[1]
encoding_dim = 32

compression_factor = float(input_dim) / encoding_dim
print("Compression factor: %s" % compression_factor)

autoencoder = Sequential()
autoencoder.add(
    Dense(encoding_dim, input_shape=(input_dim,), activation='relu')
)
autoencoder.add(
    Dense(input_dim, activation='sigmoid')
)

autoencoder.summary()

Compression factor: 24.5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_2 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [12]:
input_img = Input(shape=(input_dim,))
encoder_layer = autoencoder.layers[0]
encoder = Model(input_img, encoder_layer(input_img))

encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                25120     
Total params: 25,120
Trainable params: 25,120
Non-trainable params: 0
_________________________________________________________________


In [13]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 22s 360us/step - loss: 0.2751 - val_loss: 0.1877
Epoch 2/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.1701 - val_loss: 0.1532
Epoch 3/50
60000/60000 [==============================] - 5s 84us/step - loss: 0.1441 - val_loss: 0.1338
Epoch 4/50
60000/60000 [==============================] - 4s 61us/step - loss: 0.1286 - val_loss: 0.1216
Epoch 5/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.1190 - val_loss: 0.1140
Epoch 6/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.1127 - val_loss: 0.1088
Epoch 7/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.1079 - val_loss: 0.1045
Epoch 8/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.1039 - val_loss: 0.1009
Epoch 9/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.1007 - val_loss: 0.0980
Epo